# VacationPy
----

In [43]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [44]:
weatherDF = pd.read_csv('../WeatherPy/Output/cities.csv')
weatherDF = weatherDF.dropna()
weatherDF.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Bredasdorp,-34.5322,20.0403,53.60,82,98,6.91,ZA,1621140905
1,1,Beberibe,-4.1797,-38.1306,78.01,94,40,6.91,BR,1621140905
2,2,Bambous Virieux,-20.3428,57.7575,79.00,78,75,12.66,MU,1621140905
3,3,Kitimat,54.0524,-128.6534,51.01,88,100,2.82,CA,1621140906
4,4,Rikitea,-23.1203,-134.9692,74.80,65,92,18.32,PF,1621140644


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [28]:
#configure gmaps with API Key
gmaps.configure(api_key=g_key)

#Lat and Long as locations, humidity as weight
locations = weatherDF[["Lat","Lng"]].astype(float)
humidity = weatherDF["Humidity"].astype(float)

#Humidity heatmap
fig =gmaps.figure(zoom_level=1.9,center=(40,-15))

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False,
                                 max_intensity=100,point_radius = 2)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [45]:
#hotel_df = weatherDF[weatherDF["Max Temp"].between(60,75)& weatherDF["Wind Speed"].between(10,20) & weatherDF["Cloudiness"].between(75,100)]
perfectWeatherDF = weatherDF.loc[
    (weatherDF["Max Temp"].between(60,75)) &
    (weatherDF["Wind Speed"].between(10,20)) &
    (weatherDF["Cloudiness"].between(75,100))]
print(perfectWeatherDF.isna().values.any())
perfectWeatherDF.head()

False


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,Rikitea,-23.1203,-134.9692,74.80,65,92,18.32,PF,1621140644
39,39,Korla,41.7597,86.1469,69.30,28,98,11.72,CN,1621140920
66,66,Hasaki,35.7333,140.8333,75.00,83,75,14.97,JP,1621140931
74,74,Jiayuguan City,39.8167,98.3000,67.78,17,80,10.11,CN,1621140935
80,80,Port Alfred,-33.5906,26.8910,60.37,87,98,16.15,ZA,1621140851


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [48]:
#Store into variable named hotel_df
hotel_df = perfectWeatherDF.loc[:,['City','Lat','Lng','Country']]

#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Lat,Lng,Country,Hotel Name
4,Rikitea,-23.1203,-134.9692,PF,
39,Korla,41.7597,86.1469,CN,
66,Hasaki,35.7333,140.8333,JP,
74,Jiayuguan City,39.8167,98.3000,CN,
80,Port Alfred,-33.5906,26.8910,ZA,


In [63]:
#Set parameters to search for hotels with 5000 meters.
params = {
    "key":g_key,
    "radius":5000,
    "type":"lodging"}
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Hit the Google Places API for each city's coordinates
for index,row in hotel_df.iterrows():
    
    # update location key value
    params['location'] = f'{row["Lat"]},{row["Lng"]}'
    
     # make request
    response = requests.get(base_url,params=params).json()
    #Pull hotel Data
    try:
        hotel_df.loc[index,"Hotel Name"] = (response['results'][0]['name'])
    except:
        pass

hotel_df.head()

,City,Lat,Lng,Country,Hotel Name
4,Rikitea,-23.1203,-134.9692,PF,
39,Korla,41.7597,86.1469,CN,Gulisitan Hotel
66,Hasaki,35.7333,140.8333,JP,Hotel Sunrise Choshi
74,Jiayuguan City,39.8167,98.3000,CN,Jiugang Hotel - Jiayuguan
80,Port Alfred,-33.5906,26.8910,ZA,The Halyards Hotel


In [68]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [69]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))